In [115]:
import pandas as pd
import re
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
stop_words_en = set(stopwords.words('english'))
import matplotlib.pyplot as plt
import spacy
nlp = spacy.load('en_core_web_lg')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jyaruhernandez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jyaruhernandez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jyaruhernandez/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jyaruhernandez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/opt/homebrew/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_lg' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy valid

In [116]:
df = pd.read_csv('train.csv')

In [117]:
def funcion_preprocesamiento(texto):
    texto = re.sub(r'(#\w+)','',texto)

    texto = re.sub(r"(\w+:\/\/\S+)",'',texto)

    texto = re.sub(r'(@[A-Za-z0-9]+)','',texto)

    texto = re.sub(r'\\[A-Za-z0-9]{3}','',texto)
    texto = re.sub(r'\\n','',texto)
    texto = re.sub(r'\\','',texto)
    texto = re.sub(r'"','',texto)
    texto = re.sub(r"'",'',texto)
    texto = re.sub(r'[^\w\s]','',texto)
    texto= re.sub(r'amp','',texto)
    texto = texto.lower()
    texto = re.sub(r'([^A-Za-z \t])|(\d)','',texto)
    texto = word_tokenize(texto)
    texto = [palabra for palabra in texto if palabra not in stop_words_en]
    texto = [lemmatizer.lemmatize(palabra) for palabra in texto]
    texto = ' '.join(texto)
    return texto

In [118]:
df['clean_text'] = df['text'].apply(funcion_preprocesamiento)

In [119]:
df['vector'] = df['clean_text'].apply(lambda x: nlp(x).vector)

In [120]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          7613 non-null   int64 
 1   keyword     7552 non-null   object
 2   location    5080 non-null   object
 3   text        7613 non-null   object
 4   target      7613 non-null   int64 
 5   clean_text  7613 non-null   object
 6   vector      7613 non-null   object
dtypes: int64(2), object(5)
memory usage: 416.5+ KB


In [122]:
X = df['vector']
y = df['target']

In [123]:
X = np.concatenate(X).reshape(-1,300)

In [124]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [125]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [126]:
gradient_boosting_classifier = GradientBoostingClassifier()

In [127]:
params = {
    "learning_rate":[0.1,0.2,0.3],
    "n_estimators":[50,100,150],
    "max_depth":[3,5,8]
}

In [128]:
grid_gradiente_boosting = GridSearchCV(gradient_boosting_classifier, params, cv=10, n_jobs=-1)

In [129]:
#grid_gradiente_boosting.fit(X_train,y_train)

In [130]:
#grid_gradiente_boosting.best_params_

In [131]:
#y_pred = grid_gradiente_boosting.predict(X_test)

In [132]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report,confusion_matrix

In [133]:
#cm = confusion_matrix(y_test,y_pred)

In [134]:
#print(classification_report(y_test,y_pred))

In [135]:
import seaborn as sn

In [136]:
#sn.heatmap(cm, annot=True, annot_kws={"size": 16})

In [137]:
best_classifier = GradientBoostingClassifier(max_depth=5,learning_rate=0.1, n_estimators=150)

In [138]:
best_classifier.fit(X_train,y_train)

GradientBoostingClassifier(max_depth=5, n_estimators=150)

In [139]:
palabra = "your outfit is ablaze"

clean = funcion_preprocesamiento(palabra)

vector = nlp(clean).vector

In [140]:
vector = vector.reshape(-1,300)

In [142]:
best_classifier.predict(vector)

array([0])

In [150]:
df_submission = pd.read_csv("test.csv")

In [151]:
df_submission['clean'] = df_submission['text'].apply(funcion_preprocesamiento)

In [152]:
df_submission["vector"]  = df_submission['clean'].apply(lambda x: nlp(x).vector)

In [153]:
X_submission = df_submission["vector"]

In [155]:
X_submission = np.concatenate(X_submission).reshape(-1,300)

In [156]:
y_submission = best_classifier.predict(X_submission)

In [172]:
y_submission.shape

(3263,)

In [173]:
df_submission['id'].values.shape

(3263,)

In [174]:
data = []
for i in range(3263):
    data.append([df_submission['id'].values[i],y_submission[i]])

In [175]:
results = pd.DataFrame(data=data,columns=['id','target'])

In [177]:
results.to_csv("submission.csv", index=False)